In [ ]:
# link drive https://drive.google.com/drive/folders/18-MEX-KK80-NhISMBUj1K-AsJyr-EuX_

In [1]:
import torch
import torch.nn as nn

import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
import pandas as pd
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pickle

In [2]:
from Model import Encoder, Decoder, VAE_model
from Train import VAE_train
from LOSS import VAE_loss_function
from sklearnEval import evaluate

In [3]:
# load the data
with open('X_Y.pickle', 'rb') as handle:
    b = pickle.load(handle)

input = torch.from_numpy(b['data'].astype('float'))
test = input
input = input.float()

target = b['label']

In [4]:
DEVICE = torch.device("cpu")
batch_size = 1
x_dim  = 500
hidden_dim = 300
latent_dim = 15
lr = 1e-4

In [5]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)
model = VAE_model(Encoder=encoder, Decoder=decoder).to(DEVICE)

### Step 3. Define Loss function (reprod. loss) and optimizer

In [6]:
loss_function = VAE_loss_function


## Step 4: Train VAE **model**

In [11]:
epochs = 200
VAE_train(dataset=input, model=model, epochs = epochs, loss_=loss_function, DEVICE=DEVICE) 

Start training VAE...
	Epoch 1 complete! 	Average Loss:  0.28333547711372375
	Epoch 2 complete! 	Average Loss:  0.28296416997909546
	Epoch 3 complete! 	Average Loss:  0.28124022483825684
	Epoch 4 complete! 	Average Loss:  0.27948686480522156
	Epoch 5 complete! 	Average Loss:  0.27788829803466797
	Epoch 6 complete! 	Average Loss:  0.27598893642425537
	Epoch 7 complete! 	Average Loss:  0.2735947072505951
	Epoch 8 complete! 	Average Loss:  0.2709426283836365
	Epoch 9 complete! 	Average Loss:  0.26797497272491455
	Epoch 10 complete! 	Average Loss:  0.26468905806541443
	Epoch 11 complete! 	Average Loss:  0.26121440529823303
	Epoch 12 complete! 	Average Loss:  0.257439523935318
	Epoch 13 complete! 	Average Loss:  0.25355806946754456
	Epoch 14 complete! 	Average Loss:  0.24938252568244934
	Epoch 15 complete! 	Average Loss:  0.24507392942905426
	Epoch 16 complete! 	Average Loss:  0.2406587451696396
	Epoch 17 complete! 	Average Loss:  0.23616936802864075
	Epoch 18 complete! 	Average Loss:  0.23

In [9]:
#torch.save(model, 'model_2.pt')

In [12]:
model.eval()
zs = []
for i in input:
    _,z,_= model(i)
    zs.append(z.detach().numpy())


In [13]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(zs)
predict = kmeans.labels_
target = b['label']
evaluate(target = target, predict = predict)

Adjusted mutual info score: 0.7164904360892815
Adjusted rand score: 0.6528725959368199
Completeness score: 0.7240059245138966
Fowlkes mallow score: 0.7807572272796492
Homogeneity: 0.7097849052396215 
Completeness score: 0.7240059245138966
V measure: 0.7168248894608467
Mutual information between two clusterings score: 0.7485507913051158
Normalized Mutual Information between two clusterings score: 0.7168248894608467
Rand score: 0.8384790485888879
